In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.dropout_value = 0.1
        self.conv1 = nn.Sequential(
            nn.Conv2d(1,16,3, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(self.dropout_value)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16,32,3, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(self.dropout_value)
        )

        self.tb1= nn.Sequential(
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,8,1, padding=0, bias=False),
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(8,24,3, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(24),
            nn.Dropout(self.dropout_value)
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(24,30,3, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(30),
            nn.Dropout(self.dropout_value)
        )
        self.tb2= nn.Sequential(
            nn.MaxPool2d(2,2),
            nn.Conv2d(30,8,1, padding=0, bias=False),
        )
        self.conv5 = nn.Sequential(
            nn.Conv2d(8,16,3, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(self.dropout_value)
        )
        self.conv6 = nn.Sequential(
            nn.Conv2d(16,30,3, padding=1, bias=False),
            nn.ReLU(),
            nn.BatchNorm2d(30),
            nn.Dropout(self.dropout_value)
        )

        self.gp = nn.AvgPool2d(kernel_size=7)
        self.op =nn.Sequential(nn.Linear(30,10))


    def forward(self, x):
        x = self.conv2(self.conv1(x))
        x = self.tb1(x)
        x = self.conv4(self.conv3(x))
        x = self.tb2(x)
        x=self.conv6(self.conv5(x))
        x = self.gp(x)
        x = x.view(-1, 30)
        x = self.op(x)
        return F.log_softmax(x)

In [3]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             144
              ReLU-2           [-1, 16, 28, 28]               0
       BatchNorm2d-3           [-1, 16, 28, 28]              32
           Dropout-4           [-1, 16, 28, 28]               0
            Conv2d-5           [-1, 32, 28, 28]           4,608
              ReLU-6           [-1, 32, 28, 28]               0
       BatchNorm2d-7           [-1, 32, 28, 28]              64
           Dropout-8           [-1, 32, 28, 28]               0
         MaxPool2d-9           [-1, 32, 14, 14]               0
           Conv2d-10            [-1, 8, 14, 14]             256
           Conv2d-11           [-1, 24, 14, 14]           1,728
             ReLU-12           [-1, 24, 14, 14]               0
      BatchNorm2d-13           [-1, 24, 14, 14]              48
          Dropout-14           [-1, 24,

<ipython-input-2-8ba8af083ec2>:65: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [4]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print ("Epoch: ",epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Epoch:  1


  0%|          | 0/938 [00:00<?, ?it/s]<ipython-input-2-8ba8af083ec2>:65: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.13339091837406158 batch_id=937: 100%|██████████| 938/938 [00:22<00:00, 42.49it/s]



Test set: Average loss: 0.0724, Accuracy: 9804/10000 (98.04%)

Epoch:  2


loss=0.02977336384356022 batch_id=937: 100%|██████████| 938/938 [00:22<00:00, 41.98it/s]



Test set: Average loss: 0.0532, Accuracy: 9848/10000 (98.48%)

Epoch:  3


...


Test set: Average loss: 0.0234, Accuracy: 9923/10000 (99.23%)

Epoch:  18


loss=0.009731859900057316 batch_id=937: 100%|██████████| 938/938 [00:22<00:00, 41.62it/s]



Test set: Average loss: 0.0237, Accuracy: 9923/10000 (99.23%)

Epoch:  19


loss=0.01578695699572563 batch_id=937: 100%|██████████| 938/938 [00:23<00:00, 40.31it/s]



Test set: Average loss: 0.0298, Accuracy: 9912/10000 (99.12%)

